In [1]:
# nuclio: ignore
import nuclio

In [2]:
# Copyright 2018 Iguazio
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import pandas as pd
from mlrun.execution import MLClientCtx


def load_dataset(
        context: MLClientCtx,
        dataset: str,
        name: str = '',
        file_ext: str = 'parquet',
        params: dict = {}
) -> None:
    """Loads a scikit-learn toy dataset for classification or regression

    The following datasets are available ('name' : desription):

        'boston'          : boston house-prices dataset (regression)
        'iris'            : iris dataset (classification)
        'diabetes'        : diabetes dataset (regression)
        'digits'          : digits dataset (classification)
        'linnerud'        : linnerud dataset (multivariate regression)
        'wine'            : wine dataset (classification)
        'breast_cancer'   : breast cancer wisconsin dataset (classification)

    The scikit-learn functions return a data bunch including the following items:
    - data              the features matrix
    - target            the ground truth labels
    - DESCR             a description of the dataset
    - feature_names     header for data

    The features (and their names) are stored with the target labels in a DataFrame.

    For further details see https://scikit-learn.org/stable/datasets/index.html#toy-datasets

    :param context:    function execution context
    :param dataset:    name of the dataset to load
    :param name:       artifact name (defaults to dataset)
    :param file_ext:   output file_ext: parquet or csv
    :param params:     params of the sklearn load_data method
    """
    dataset = str(dataset)
    # reach into module and import the appropriate load_xxx function
    pkg_module = 'sklearn.datasets'
    fname = f'load_{dataset}'

    pkg_module = __import__(pkg_module, fromlist=[fname])
    load_data_fn = getattr(pkg_module, fname)

    data = load_data_fn(**params)
    feature_names = data['feature_names']

    # create the toy dataset
    xy = np.concatenate([data['data'], data['target'].reshape(-1, 1)], axis=1)
    if hasattr(feature_names, 'append'):
        # its a list
        feature_names.append('labels')
    else:
        # its an array
        feature_names = np.append(feature_names, 'labels')
    df = pd.DataFrame(data=xy, columns=feature_names)

    # log and upload the dataset
    context.log_dataset(name or dataset, df=df, format=file_ext, index=False)

In [3]:
# nuclio: end-code

### mlconfig

In [4]:
from mlrun import mlconf

In [5]:
mlconf.dbpath = mlconf.dbpath or './'
mlconf.dbpath

'http://mlrun-api:8080'

In [6]:
vcs_branch = 'development'
base_vcs = f'https://raw.githubusercontent.com/mlrun/functions/{vcs_branch}/'

mlconf.hub_url = mlconf.hub_url or base_vcs + f'{name}/function.yaml'
mlconf.hub_url

'/User/repos/functions/{name}/function.yaml'

In [7]:
import os
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["V3IO_HOME"]}/artifacts'
mlconf.artifact_path

'/User/artifacts'

### save

In [8]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dataset", kind="job", with_doc=True,
                      handler=load_dataset, 
                      image="mlrun/ml-models")

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dataset"
fn.spec.description = "load a toy dataset from scikit-learn"
fn.metadata.categories = ["datagen", "filesutils"]
fn.metadata.labels = {"author": "yjb"}

fn.save()
fn.export("function.yaml")

[mlrun] 2020-04-25 14:28:57,478 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-25 14:28:57,518 function spec saved to path: function.yaml


## tests

In [9]:
from mlrun import import_function

func = import_function("hub://load_dataset")

if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    func.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    func.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

[mlrun] 2020-04-25 14:28:57,534 warning!, server (0.4.6) and client (0.4.7) ver dont match


In [10]:
from mlrun import NewTask    

task_params = {
    "name"   : "tasks load toy dataset", 
    "params" : {"dataset"     : "wine"}}

### run remotely

In [11]:
run = func.run(NewTask(**task_params))

[mlrun] 2020-04-25 14:28:57,575 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-25 14:28:57,576 starting run tasks load toy dataset uid=e1ad21ee582a4bba8da1ae83b390a874  -> http://mlrun-api:8080
[mlrun] 2020-04-25 14:28:57,700 Job is running in the background, pod: tasks-load-toy-dataset-ghkbv
[mlrun] 2020-04-25 14:30:05,368 log artifact wine at /User/artifacts/wine.parquet, size: 17997, db: Y

[mlrun] 2020-04-25 14:30:05,409 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...90a874,0,Apr 25 14:30:05,completed,tasks load toy dataset,host=tasks-load-toy-dataset-ghkbvkind=jobowner=adminv3io_user=admin,,dataset=wine,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run e1ad21ee582a4bba8da1ae83b390a874  , !mlrun logs e1ad21ee582a4bba8da1ae83b390a874 
[mlrun] 2020-04-25 14:30:07,186 run executed, status=completed


### or locally

In [12]:
from mlrun import run_local

In [13]:
for dataset in ["wine", "iris", "breast_cancer"]:
    run_local(handler=load_dataset,
              inputs={"dataset": dataset})

[mlrun] 2020-04-25 14:30:07,210 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-25 14:30:07,210 starting run mlrun-a4a74c-load_dataset uid=f8a065b52e85473e8250d7dfd8209773  -> http://mlrun-api:8080
[mlrun] 2020-04-25 14:30:07,228 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-25 14:30:09,109 log artifact wine at /User/artifacts/wine.parquet, size: 18006, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...209773,0,Apr 25 14:30:07,completed,mlrun-a4a74c-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-5859859b4f-bjqf5,dataset,,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run f8a065b52e85473e8250d7dfd8209773 --project default , !mlrun logs f8a065b52e85473e8250d7dfd8209773 --project default
[mlrun] 2020-04-25 14:30:09,174 run executed, status=completed
[mlrun] 2020-04-25 14:30:09,183 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-25 14:30:09,183 starting run mlrun-0d4c95-load_dataset uid=8d88624036c14436933dc54eded373f3  -> http://mlrun-api:8080
[mlrun] 2020-04-25 14:30:09,202 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-25 14:30:09,271 log artifact iris at /User/artifacts/iris.parquet, size: 5049, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...d373f3,0,Apr 25 14:30:09,completed,mlrun-0d4c95-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-5859859b4f-bjqf5,dataset,,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run 8d88624036c14436933dc54eded373f3 --project default , !mlrun logs 8d88624036c14436933dc54eded373f3 --project default
[mlrun] 2020-04-25 14:30:09,318 run executed, status=completed
[mlrun] 2020-04-25 14:30:09,325 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-25 14:30:09,326 starting run mlrun-4f8af7-load_dataset uid=7e1f5420cc0a428e93649dd467913944  -> http://mlrun-api:8080
[mlrun] 2020-04-25 14:30:09,351 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-25 14:30:09,554 log artifact breast_cancer at /User/artifacts/breast_cancer.parquet, size: 146754, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...913944,0,Apr 25 14:30:09,completed,mlrun-4f8af7-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-5859859b4f-bjqf5,dataset,,,breast_cancer


to track results use .show() or .logs() or in CLI: 
!mlrun get run 7e1f5420cc0a428e93649dd467913944 --project default , !mlrun logs 7e1f5420cc0a428e93649dd467913944 --project default
[mlrun] 2020-04-25 14:30:09,635 run executed, status=completed
